In [2]:
import pprint

Below is intended to add guidance for evaluate orchestrator-based bot.


## 1) load orchestrator-based bot response

1. visit https://devbot-api.nvidia.com/evaluation/docs#/, then use chat/completions to get full response (example curl attached below)
2. check response and add extra handling logic in "service_library.run.parser_library.response_parser". 
3. update schema in "RegressionSchema" -> "RunConfig" -> "Outputs", note that the order matters as the input is processed in sequencial matter.

Here are the details for schema

- Type: Attribute

value (String) : keys are seperated by dot ("."), will be derived by `dict.get(<key>)` or `list[<key>]`

- Type: Function

value (String): function in directories in repo, seperated by dot (".")
args (List): each argument to be used in function, each item in list is the key of *current* dictionary


1)
To view response

```
curl --location 'http://localhost:<port>/evaluation/chat/completions_preprocessing' \
--header 'accept: application/json' \
--header 'Content-Type: application/json' \
--data '{
  "model": "nvbot-adapter",
  "messages": [
    {
      "role": "system",
      "content": "A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human'\''s questions."
    },
    {
      "role": "user",
      "content": "How is my Subscription Date and Price determined?"
    }
  ],
  "max_tokens": 25,
  "temperature": 0,
  "top_p": 0,
  "stop": null,
  "user": "",
  "extra_body": {
    "project_id": 11,
    "env": "sandbox",
    "system": "nvinfo",
    "model": "mixtral_agent"
  }
}
'
```


2) to extract response
- for simple response extraction,

Outputs:
```
[
        {
          "name": "Response",
          "type": "Attribute",
          "value": ""
        },
        {
          "name": "Response",
          "type": "Function",
          "value": "service_library.run.parser_library.response_parser.get_orchestrator_response",
          "args": [
            "Response"
          ]
        }
      ]
```


- for complex response and post-processing logic
add functions as part of response_parser (or define your own response)

Outputs:
```
[
        {
          "name": "Response",
          "type": "Attribute",
          "value": ""
        },
        {
          "name": "Debug response",
          "type": "Attribute",
          "value": "-1.-1.debug_logger.-1"
        },
        {
          "type": "Function",
          "value": "service_library.run.parser_library.response_parser.decompose_stringify_json_data",
          "args": [
            "Debug response"
          ]
        },
        {
          "name": "Response",
          "type": "Function",
          "value": "service_library.run.parser_library.response_parser.get_orchestrator_response",
          "args": [
            "Response"
          ]
        },
        {
          "type": "Function",
          "value": "service_library.run.parser_library.response_parser.response_citation_parser"
        }
      ]
```
MR changes required if a new state needs to be added to bot for extra logging:

[nvinfo](https://gitlab-master.nvidia.com/jarvis-chatbot/nvinfo-orchestrator-agent/-/merge_requests/123/diffs#3c42c999b2c16f4b347680f52935248420c82725)
[config manager](https://gitlab-master.nvidia.com/jarvis-chatbot/langchain-config-selector/-/merge_requests/495/diffs)


curl
```
curl --location 'http://localhost:<port>/evaluation/chat/completions_postprocessing' \
--header 'accept: application/json' \
--header 'Content-Type: application/json' \
--data '{
    "model": "nvbot-adapter",
    "messages": [
        {
            "role": "system",
            "content": "A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human'\''s questions."
        },
        {
            "role": "user",
            "content": "What is NVIDIA core values?"
        }
    ],
    "max_tokens": 25,
    "temperature": 0,
    "top_p": 0,
    "stop": null,
    "user": "",
    "extra_body": {
        "env": "stg",
        "system": "nvinfo",
        "model": "mixtral_agent",
        "userid": "nvbot-evaluation"
    },
    "output_schema":        [
     {
          "name": "Response",
          "type": "Attribute",
          "value": ""
        },
        {
          "name": "Debug response",
          "type": "Attribute",
          "value": "-1.-1.debug_logger.-1"
        },
        {
          "type": "Function",
          "value": "service_library.run.parser_library.response_parser.decompose_stringify_json_data",
          "args": [
            "Debug response"
          ]
        },
        {
          "name": "Response",
          "type": "Function",
          "value": "service_library.run.parser_library.response_parser.get_orchestrator_response",
          "args": [
            "Response"
          ]
        },
        {
          "type": "Function",
          "value": "service_library.run.parser_library.response_parser.response_citation_parser"
        }
    ]
}'
```

Note that `output_schema` should be used as "Outputs", where in schema path as "RegressionSchema" -> "RunConfig" -> "Outputs"

 example response output for above NVInfo example
![nvinfo output](images/processed_output.png)